In [ ]:
!pip install seaborn
!pip install scikit-learn
!pip install matplotlib
!pip install shap
!pip install plotly

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import snowflake.snowpark.functions as F 
from snowflake.ml.modeling.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from snowflake.ml.modeling.xgboost import XGBClassifier
import shap
import plotly.graph_objects as go
from datetime import timedelta
from snowflake.ml.registry import Registry

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"Player_360", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}

In [ ]:
USE ROLE SYSADMIN;

In [ ]:
session.sql("""CREATE OR REPLACE WAREHOUSE so_warehouse WITH
  WAREHOUSE_SIZE = 'LARGE'
  WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED'
  RESOURCE_CONSTRAINT = 'MEMORY_16X_X86';
""").collect()     

In [ ]:
USE WAREHOUSE so_warehouse;

## Rolling Predictions Churn Model

In [ ]:
# gather session information, point per session information, and purchase information
session_points_df = session.sql("""
SELECT 
s.session_id,
s.user_id,
s.log_in,
s.session_duration_minutes,
s.device_type,
ppe.total_points AS total_points_per_session
FROM PLAYER_360.RAW.SESSIONS s 
LEFT JOIN PLAYER_360.ANALYTIC.POINTS_PER_EVENT ppe ON s.session_id = ppe.session_id
""").to_pandas()
session_points_df[:100]

In [ ]:
session_points_df['DEVICE_TYPE'].value_counts()

In [ ]:
session_points_df["DAY"] = pd.to_datetime(session_points_df["LOG_IN"].dt.date)
# Sort the dataframe by USER_ID and LOG_IN to ensure the rolling window works properly
session_points_df = session_points_df.sort_values(by=['USER_ID', 'DAY', 'LOG_IN'])
df = session_points_df
len(df)

In [ ]:
# set the window size
window = 30

In [ ]:
day_sessions_df = df.groupby(['USER_ID','DAY']).agg(
    total_session_duration=('SESSION_DURATION_MINUTES', 'sum'),
    total_sessions=('SESSION_ID', 'count'),
    total_points=('TOTAL_POINTS_PER_SESSION', 'sum')
).reset_index()

# for each user add in the days they were active as 0 and inactive as 1
day_sessions_df['SESSION_INACTIVE'] = 0

day_sessions_df.columns = [u.upper() for u in list(day_sessions_df.columns)]
len(day_sessions_df)

In [ ]:
# extract the first and last login days for each user to fill the date range 
# allows for faster computation since we only ahve to go 30 indices back instead of checking if each
# DATE object is within 30 days
users_logins_df = df.groupby('USER_ID').agg(
    first_login_day=('DAY', 'first'),
    last_login_day=('DAY', 'last')
).reset_index()
users_logins_df.columns = [u.upper() for u in list(users_logins_df.columns)]
len(users_logins_df)

In [ ]:
# holds per user, the date range filled sessions information
full_day_sessions = []

# for each user, fill the date range for that user and merge back into day_sessions_df
for _, row in users_logins_df.iterrows():
    user_id = row['USER_ID']
    start_date = row['FIRST_LOGIN_DAY']
    end_date = row['LAST_LOGIN_DAY']
   
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    user_days_df = pd.DataFrame({'USER_ID': user_id, 'DAY': date_range})
    user_day_sessions = pd.merge(user_days_df, day_sessions_df[day_sessions_df['USER_ID'] == user_id], 
                                 on=['USER_ID', 'DAY'], how='left')

    user_day_sessions['TOTAL_SESSION_DURATION'] = user_day_sessions['TOTAL_SESSION_DURATION'].fillna(0)
    user_day_sessions['TOTAL_SESSIONS'] = user_day_sessions['TOTAL_SESSIONS'].fillna(0)
    user_day_sessions['TOTAL_POINTS'] = user_day_sessions['TOTAL_POINTS'].fillna(0)
    user_day_sessions['SESSION_INACTIVE'] = user_day_sessions['SESSION_INACTIVE'].fillna(1)
    
    full_day_sessions.append(user_day_sessions)

day_sessions_df = pd.concat(full_day_sessions).reset_index(drop=True)
len(day_sessions_df)

In [ ]:
day_sessions_df['TOTAL_SESSION_DURATION_ROLLING_30_DAYS'] = day_sessions_df.groupby('USER_ID')['TOTAL_SESSION_DURATION'].rolling(
    window=window, min_periods=1).sum().reset_index(level=0,drop=True)
day_sessions_df['TOTAL_SESSIONS_ROLLING_30_DAYS'] = day_sessions_df.groupby('USER_ID')['TOTAL_SESSIONS'].rolling(
    window=window, min_periods=1).sum().reset_index(level=0,drop=True)
day_sessions_df['AVERAGE_SESSION_LEN_ROLLING_30_DAYS'] = day_sessions_df['TOTAL_SESSION_DURATION_ROLLING_30_DAYS'] / day_sessions_df['TOTAL_SESSIONS_ROLLING_30_DAYS']
day_sessions_df['TOTAL_POINTS_ROLLING_30_DAYS'] = day_sessions_df.groupby('USER_ID')['TOTAL_POINTS'].rolling(
    window=window, min_periods=1).sum().reset_index(level=0,drop=True)
day_sessions_df['AVERAGE_POINTS_PER_SESSION_ROLLING_30_DAYS'] = day_sessions_df['TOTAL_POINTS_ROLLING_30_DAYS'] / day_sessions_df['TOTAL_SESSIONS_ROLLING_30_DAYS']
len(day_sessions_df)

In [ ]:
rolling_sessions_df = day_sessions_df[['USER_ID', 'DAY', 'SESSION_INACTIVE', \
                                       'TOTAL_SESSION_DURATION_ROLLING_30_DAYS', \
                                      'TOTAL_SESSIONS_ROLLING_30_DAYS', \
                                      'AVERAGE_SESSION_LEN_ROLLING_30_DAYS', \
                                      'TOTAL_POINTS_ROLLING_30_DAYS', \
                                      'AVERAGE_POINTS_PER_SESSION_ROLLING_30_DAYS']]
len(rolling_sessions_df)

In [ ]:
def remove_first_30_days(df):
    df = df.sort_values(by=['USER_ID', 'DAY'])
    df = df.groupby('USER_ID').apply(lambda x: x.iloc[30:]).reset_index(drop=True)
    return df

In [ ]:
# remove the first 29 days metrics for each user_id because we want full 30 day averages
rolling_sessions_df = remove_first_30_days(rolling_sessions_df)
len(rolling_sessions_df)

In [ ]:
user_1001_session_information = rolling_sessions_df[rolling_sessions_df['USER_ID'] == 1001]
plt.figure(figsize=(10, 6))
plt.plot(user_1001_session_information['DAY'], 
         user_1001_session_information['TOTAL_SESSION_DURATION_ROLLING_30_DAYS'], label='Total Session Duration (30 days)', color='b', linestyle='-', marker='o')
plt.plot(user_1001_session_information['DAY'], 
         user_1001_session_information['TOTAL_SESSIONS_ROLLING_30_DAYS'], label='Total Sessions (30 days)', color='g', linestyle='-', marker='x')
plt.plot(user_1001_session_information['DAY'], 
         user_1001_session_information['AVERAGE_SESSION_LEN_ROLLING_30_DAYS'], label='Average Session Length (30 days)', color='r', linestyle='-', marker='s')
plt.plot(user_1001_session_information['DAY'], 
         user_1001_session_information['TOTAL_POINTS_ROLLING_30_DAYS'], label='Total Points (30 days)', color='c', linestyle='-', marker='d')
plt.plot(user_1001_session_information['DAY'], 
         user_1001_session_information['AVERAGE_POINTS_PER_SESSION_ROLLING_30_DAYS'], label='Average Points per Session (30 days)', color='m', linestyle='-', marker='^')

inactive_mask = user_1001_session_information['SESSION_INACTIVE'] == 1
start_day = None
end_day = None

# Plot shaded regions for inactivity periods
for i in range(1, len(user_1001_session_information)):
    if inactive_mask[i] and not inactive_mask[i-1]:
        # Start of inactivity
        start_day = user_1001_session_information['DAY'].iloc[i]
    elif not inactive_mask[i] and inactive_mask[i-1]:
        # End of inactivity
        end_day = user_1001_session_information['DAY'].iloc[i-1]
        # Ensure both start_day and end_day are defined before plotting
        if start_day is not None and end_day is not None:
            plt.axvspan(start_day, end_day, color='gray', alpha=0.3, label='Inactive Period' if i == 1 else "")
        start_day = None  # Reset start_day after plotting

plt.xlabel('Day')
plt.ylabel('Value')
plt.yscale('log')
plt.title('Rolling Metrics for USER_ID 1001 (30 days)')
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# full dataset of all ads
purchases_df = session.table("PLAYER_360.RAW.PURCHASES").to_pandas()
purchases_df.head()

In [ ]:
# dataset only of ads that lead to purchases
purchased_df = purchases_df[purchases_df['PURCHASE_TYPE'] != 'none']
purchased_df.head()

In [ ]:
# get aggregate metrics by day as intermediary to calculate 30 day rolling metrics
purchases_df['DAY'] = pd.to_datetime(purchases_df['TIMESTAMP_OF_PURCHASE'].dt.date)
day_purchases_df = purchases_df.groupby(['USER_ID', 'DAY']).agg(
    total_ad_engagement_time=('AD_ENGAGEMENT_TIME', 'sum'),
    total_ad_conversions=('AD_CONVERSION', 'sum'),
    total_ads=('AD_INTERACTION_ID', 'count')
).reset_index()
day_purchases_df['PURCHASE_INACTIVE'] = 0
# merge in day_sessions_df to ensure consistency with date_ranges
day_purchases_df = pd.merge(day_sessions_df, day_purchases_df, how="left")[list(day_purchases_df.columns)]
day_purchases_df[list(day_purchases_df.columns)[:-1]] = day_purchases_df[list(day_purchases_df.columns)[:-1]].fillna(0)
day_purchases_df['PURCHASE_INACTIVE'] = day_purchases_df['PURCHASE_INACTIVE'].fillna(1)
day_purchases_df.head()

In [ ]:
# get aggregate metrics by day as intermediary to calculate 30 day rolling metrics for only purchases
purchased_df['DAY'] = pd.to_datetime(purchased_df['TIMESTAMP_OF_PURCHASE'].dt.date)
day_purchased_df = purchased_df.groupby(['USER_ID', 'DAY']).agg(
    total_purchase_amount=('PURCHASE_AMOUNT', 'sum'),
    average_purchase_amount=('PURCHASE_AMOUNT', 'mean'),
    total_purchases = ('PURCHASE_ID', 'count')
).reset_index()
day_purchased_df.head()

In [ ]:
# now perform final ad and purchase merge
result_df = pd.merge(day_purchases_df, day_purchased_df, on=['USER_ID', 'DAY'], how='left').fillna(0)
result_df.columns = [u.upper() for u in result_df.columns]
result_df.head()

In [ ]:
result_df['TOTAL_PURCHASE_AMOUNT_ROLLING_30_DAYS'] = result_df.groupby('USER_ID')['TOTAL_PURCHASE_AMOUNT'].rolling(window, min_periods=1).sum().reset_index(level=0, drop=True)
result_df['TOTAL_PURCHASES_ROLLING_30_DAYS'] = result_df.groupby('USER_ID')['TOTAL_PURCHASES'].rolling(window, min_periods=1).sum().reset_index(level=0, drop=True)
result_df['AVG_PURCHASE_AMOUNT_ROLLING_30_DAYS'] = result_df['TOTAL_PURCHASE_AMOUNT_ROLLING_30_DAYS'] / result_df['TOTAL_PURCHASES_ROLLING_30_DAYS']
result_df['TOTAL_ADS_ROLLING_30_DAYS'] = result_df.groupby('USER_ID')['TOTAL_ADS'].rolling(window, min_periods=1).sum().reset_index(level=0, drop=True)
result_df['AD_CONVERSION_RATE_ROLLING_30_DAYS'] = result_df['TOTAL_PURCHASES_ROLLING_30_DAYS'] /result_df['TOTAL_ADS_ROLLING_30_DAYS']
result_df['TOTAL_AD_ENGAGEMENT_TIME_ROLLING_30_DAYS'] = result_df.groupby('USER_ID')['TOTAL_AD_ENGAGEMENT_TIME'].rolling(window, min_periods=1).sum().reset_index(level=0, drop=True)
result_df['AVERAGE_ENGAGEMENT_TIME_ROLLING_30_DAYS'] = result_df['TOTAL_AD_ENGAGEMENT_TIME_ROLLING_30_DAYS'] /result_df['TOTAL_ADS_ROLLING_30_DAYS']
result_df['AVG_PURCHASE_AMOUNT_ROLLING_30_DAYS'] = result_df['AVG_PURCHASE_AMOUNT_ROLLING_30_DAYS'].fillna(0)
result_df['AD_CONVERSION_RATE_ROLLING_30_DAYS'] = result_df['AD_CONVERSION_RATE_ROLLING_30_DAYS'].fillna(0)
result_df['AVERAGE_ENGAGEMENT_TIME_ROLLING_30_DAYS'] = result_df['AVERAGE_ENGAGEMENT_TIME_ROLLING_30_DAYS'].fillna(0)
result_df.head(100)

In [ ]:
# drop the per day metrics and keep only rolling 30 day metrics
rolling_purchases_df = result_df[['USER_ID', 'DAY', 'PURCHASE_INACTIVE', \
                                  'TOTAL_PURCHASE_AMOUNT_ROLLING_30_DAYS', \
                                 'TOTAL_PURCHASES_ROLLING_30_DAYS', \
                                 'AVG_PURCHASE_AMOUNT_ROLLING_30_DAYS', \
                                 'TOTAL_ADS_ROLLING_30_DAYS', \
                                 'AD_CONVERSION_RATE_ROLLING_30_DAYS', \
                                 'TOTAL_AD_ENGAGEMENT_TIME_ROLLING_30_DAYS', \
                                 'AVERAGE_ENGAGEMENT_TIME_ROLLING_30_DAYS']]
rolling_purchases_df.head()

In [ ]:
len(rolling_purchases_df)

In [ ]:
rolling_purchases_df = remove_first_30_days(rolling_purchases_df)
len(rolling_purchases_df)

In [ ]:
# merge sessions and purchases information to have final features dataframe for model training
features_df = pd.merge(rolling_sessions_df, rolling_purchases_df, on=["USER_ID","DAY"], how="outer")
features_df.head()

In [ ]:
features_df['USER_ID'] = features_df['USER_ID'].astype('int32')

binary_list = ['SESSION_INACTIVE', 'PURCHASE_INACTIVE']
features_df[binary_list] = features_df[binary_list].astype('int8')

integer_columns = [u for u in features_df.columns if 'TOTAL' in u] + ['USER_ID'] 
integer_columns = [col for col in integer_columns  if 'TOTAL_POINTS_ROLLING_30_DAYS' != col] 
integer_columns
features_df[integer_columns] = features_df[integer_columns].astype('int32')

float_columns = set(features_df.columns) - set(integer_columns) - set(binary_list) - set(['DAY'])
features_df[list(float_columns)] = features_df[list(float_columns)].astype('float32')

In [ ]:
# Function to compute whether a user logged in in the next 7 days, used for churn labeling
def calculate_login_within_7_days(user_data):
    # Sum the 'SESSION_INACTIVE' values for the next 7 days (excluding current day)
    user_data.loc[:, 'future_sessions_sum'] = user_data['SESSION_INACTIVE'].shift(-7).rolling(window=7, min_periods=1).sum()

    # If the sum is less than 7, then the user logged in within the next 7 days
    user_data.loc[:, 'LOGIN_NEXT_7_DAYS'] = (user_data['future_sessions_sum'] < 7).astype(int)

    # Drop the temporary 'future_sessions_sum' column, if not needed
    user_data.drop(columns=['future_sessions_sum'], inplace=True)

    return user_data

# Sort the data by USER_ID and DAY
features_df = features_df.sort_values(by=['USER_ID', 'DAY'])


In [ ]:
features_df['LOGIN_NEXT_7_DAYS'] = 0
features_df['LOGIN_NEXT_7_DAYS'] = features_df['LOGIN_NEXT_7_DAYS'].astype(int)
features_df['LOGIN_NEXT_7_DAYS'] = features_df.groupby('USER_ID', group_keys=False).apply(calculate_login_within_7_days)['LOGIN_NEXT_7_DAYS']
features_df.head(100)

In [ ]:
# remove the currently active users from the dataset as the users to predict
retention_df = session.table("PLAYER_360.ANALYTIC.RETENTION").to_pandas()
active_users = retention_df[retention_df['CHURNED'] == 0]
df1_filtered = features_df[features_df['USER_ID'].isin(active_users['USER_ID'])]
df1_filtered = df1_filtered.sort_values(by=['USER_ID', 'DAY'])
to_pred_df = df1_filtered.groupby('USER_ID').apply(lambda x: x.iloc[-1]).reset_index(drop=True)

# this is the dataset to predicted with our final trained and tested model
to_pred_df.head()

In [ ]:
# remove from features this dataset of currently active users
mask = features_df[['USER_ID', 'DAY']].isin(to_pred_df[['USER_ID', 'DAY']]).all(axis=1)
features_df = features_df[~mask]
len(features_df)

In [ ]:
final_features_df = features_df.drop(labels=['USER_ID', 'DAY', 'SESSION_INACTIVE', 'PURCHASE_INACTIVE'], axis=1)
final_features_df.head()

# EDA

In [ ]:
final_features_df.describe()

In [ ]:
sns.heatmap(final_features_df[list(final_features_df.describe())].corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
y = features_df['LOGIN_NEXT_7_DAYS']
churned_data = final_features_df[y == 0]
non_churned_data = final_features_df[y == 1]
zero_class_count = len(churned_data)
one_class_count = len(non_churned_data)
print(zero_class_count)
print(one_class_count)

# Model Training

In [ ]:
# save the dataset as ROLLING_CHURN_FEATURES
final_features_df = session.write_pandas(df=features_df.reset_index(), \
                     table_name="ROLLING_CHURN_FEATURES", database="PLAYER_360", schema="APP", \
                     quote_identifiers=False,
                     auto_create_table=True,
                     overwrite=True)

In [ ]:
to_pred_df = session.write_pandas(df=to_pred_df.reset_index(), \
                     table_name="TO_BE_PREDICTED_CHURN_FEATURES", database="PLAYER_360", schema="APP", \
                     quote_identifiers=False,
                     auto_create_table=True,
                     overwrite=True).to_pandas()

In [ ]:
final_features_df = session.table("PLAYER_360.APP.ROLLING_CHURN_FEATURES")

In [ ]:
# split the dataset into test and train
training, testing = final_features_df.random_split(weights=[0.8, 0.2], seed=111)

In [ ]:
Target = ['LOGIN_NEXT_7_DAYS']
Output_label = ['PREDICTED_CHURN']
Features_label = [
    "TOTAL_SESSION_DURATION_ROLLING_30_DAYS",
    "TOTAL_SESSIONS_ROLLING_30_DAYS",
    "AVERAGE_SESSION_LEN_ROLLING_30_DAYS",
    "TOTAL_POINTS_ROLLING_30_DAYS",
    "AVERAGE_POINTS_PER_SESSION_ROLLING_30_DAYS",
    "TOTAL_PURCHASE_AMOUNT_ROLLING_30_DAYS",
    "TOTAL_PURCHASES_ROLLING_30_DAYS",
    "AVG_PURCHASE_AMOUNT_ROLLING_30_DAYS",
    "TOTAL_ADS_ROLLING_30_DAYS",
    "AD_CONVERSION_RATE_ROLLING_30_DAYS",
    "TOTAL_AD_ENGAGEMENT_TIME_ROLLING_30_DAYS",
    "AVERAGE_ENGAGEMENT_TIME_ROLLING_30_DAYS"
]

model = XGBClassifier(
    input_cols=Features_label,
    label_cols=Target,
    output_cols=Output_label,
    scale_pos_weight= zero_class_count/one_class_count, # replace with zero_class_count/ one_class_count
)

In [ ]:
training_df = training.to_pandas()
model.fit(training_df)

In [ ]:
testing_df =testing.to_pandas()
predictions = model.predict_proba(testing_df)
true_labels = testing_df['LOGIN_NEXT_7_DAYS']
churn_likelihood = predictions[['PREDICT_PROBA_0','PREDICT_PROBA_1']]
churn_likelihood.head()

In [ ]:
# if the anamolous class (0) has probability >= .5, then didn't login in next 7 days
predicted_churn = [0 if p >= .4 else 1 for p in churn_likelihood['PREDICT_PROBA_0']]
# Compute confusion matrix
cm = confusion_matrix(true_labels, predicted_churn)
print("Confusion Matrix:")
print(cm)

# Generate classification report
report = classification_report(true_labels, predicted_churn)
print("Classification Report:")
print(report)

# Log the Model in the Model Registry

In [ ]:
USE SCHEMA APP;

In [ ]:
from snowflake.ml.registry import Registry

reg = Registry(session=session)

MODEL_NAME = "Player360_RollingChurn_Classifier"
MODEL_VERSION = "v1"

X_train = training_df[Features_label][:1000]

mv = reg.log_model(model,
                   model_name=MODEL_NAME,
                   version_name=MODEL_VERSION,
                   options={
                       "case_sensitive": True,
                       "enable_explainability": True
    },
                    sample_input_data=X_train,
                    target_platforms=["WAREHOUSE"]
)

mv.set_metric("confusion_matrix", cm.tolist())
mv.set_metric("classification_report", report)

mv.comment = "This is the first iteration of our Rolling Churn Classification model."

In [ ]:
reg.show_models()

In [ ]:
X_test = testing.select(Features_label).limit(10)
mv.run(X_test, function_name="predict_proba")

# Model Explainability

In [ ]:
mv_explanations = mv.run(X_test, function_name="explain")
mv_explanations =mv_explanations.to_pandas()
mv_explanations.head()

In [ ]:
shap_exp = shap._explanation.Explanation(mv_explanations.values, feature_names=mv_explanations.columns)
shap.plots.bar(shap_exp)